# SALES ANALYSIS OF TECH ITEMS IN 2019

We have a dataset with tech products that were sold and we need to reply to the following business questions:



1.   Which city has highest number of sales?
2.   Which month has highest number of sold items?
3.   When is the best time to show ads in order to increase customer likelihood of buying product?
4.   What items are commonly sold together?

.

.

Importing the required libraries:

In [155]:
from google.colab import drive
import os
import pandas as pd
import plotly.express as px
from collections import Counter

.

In [156]:
# Mounting Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [157]:
# Setting folder path
folder_path = '/content/drive/MyDrive/DATA_ANALYSIS_PROJECTS/Sales_Data_Analysis_-_Project/Data'

In [158]:
# Using os.listdir to get a list of all files and subfolders
folder_contents = os.listdir(folder_path)

# Printing the contents of the folder
for item in folder_contents:
    print(item)

Sales_April_2019.csv
Sales_March_2019.csv
Sales_January_2019.csv
Sales_May_2019.csv
Sales_February_2019.csv
Sales_September_2019.csv
Sales_August_2019.csv
Sales_July_2019.csv
Sales_June_2019.csv
Sales_October_2019.csv
Sales_November_2019.csv
Sales_December_2019.csv


We have separate files for each month, so we need to start by merging them into a single dataframe.

In [159]:
df = pd.DataFrame()

for item in folder_contents:
  item_df = pd.read_csv(os.path.join(folder_path, item))
  df = pd.concat([df, item_df], ignore_index=True)

In [160]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


We can see that there are NaN values but there is no difference in the number of Nan between different columns. We can therefore imagine that we are dealing with NaN rows.

In [162]:
# Dropping rows where all values are NaN
df = df.dropna(how='all')

Now we are going to check for invalid cells that contain non numerical characters.

In [163]:
columns_to_check = ['Order ID', 'Quantity Ordered', 'Price Each', 'Order Date']

# Set for storing indices of invalid rows
invalid_row_indices = set()

# Allowed characters: digits, space, point, '/' and ':'
allowed_chars = set('0123456789 ./:')

# Iterating through each element in the DataFrame
for col in df[columns_to_check]:
    for index, item in df[col].items():
        # Check if each character in the string is allowed
        if not all(char in allowed_chars for char in str(item)):
            invalid_row_indices.add(index)

# Retrieving rows with invalid data using the collected indices
invalid_row_indices = list(invalid_row_indices)
invalid_rows = df.loc[invalid_row_indices]

print(invalid_rows)

        Order ID  Product  Quantity Ordered  Price Each  Order Date  \
20490   Order ID  Product  Quantity Ordered  Price Each  Order Date   
61450   Order ID  Product  Quantity Ordered  Price Each  Order Date   
90123   Order ID  Product  Quantity Ordered  Price Each  Order Date   
182282  Order ID  Product  Quantity Ordered  Price Each  Order Date   
75793   Order ID  Product  Quantity Ordered  Price Each  Order Date   
...          ...      ...               ...         ...         ...   
98279   Order ID  Product  Quantity Ordered  Price Each  Order Date   
85993   Order ID  Product  Quantity Ordered  Price Each  Order Date   
34803   Order ID  Product  Quantity Ordered  Price Each  Order Date   
174069  Order ID  Product  Quantity Ordered  Price Each  Order Date   
36856   Order ID  Product  Quantity Ordered  Price Each  Order Date   

        Purchase Address  
20490   Purchase Address  
61450   Purchase Address  
90123   Purchase Address  
182282  Purchase Address  
75793   Purc

In [164]:
# Dropping the rows with invalid elements
df = df.drop(invalid_row_indices)

In [165]:
# Converting the columns to the correct type

for col in columns_to_check:
  if col == 'Order Date':
    df[col] = pd.to_datetime(df[col], errors='coerce')
  else:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185950 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Order ID          185950 non-null  int64         
 1   Product           185950 non-null  object        
 2   Quantity Ordered  185950 non-null  int64         
 3   Price Each        185950 non-null  float64       
 4   Order Date        185950 non-null  datetime64[ns]
 5   Purchase Address  185950 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 9.9+ MB


The dataframe is now ready and we can move on to reply to the business questions.

.

.

# Business Question n1: Which city has the higher number of sales?

First thing we need to do some feature engineering: we are going to create a "Sales" column by multiplying Quantity Ordered * Price Each.

In [167]:
df['Sales'] = df['Quantity Ordered'] * df['Price Each']

And then to create a City column by extracting the city name from the whole address string.

In [168]:
df['City'] = df['Purchase Address'].apply(lambda x: x.split(',')[1])

.

Checking the result:

In [169]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Sales,City
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",23.90,Dallas
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",99.99,Boston
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",600.00,Los Angeles
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",11.99,Los Angeles
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",11.99,Los Angeles


Finding the answer:

In [170]:
# Grouping by city
df_by_city = df.groupby('City')['Sales'].sum()

max_sales_city = df_by_city.idxmax()
max_sales_value = df_by_city.max()

print(f"The city with the highest sales is{max_sales_city} with sales of {max_sales_value} USD")

The city with the highest sales is San Francisco with sales of 8262203.91 USD


.

In [171]:
# Resetting the index to turn 'City' into a regular column
df_by_city = df_by_city.reset_index()

# Sorting the DataFrame by 'Sales' in descending order
df_by_city_sorted = df_by_city.sort_values(by='Sales', ascending=False)

# Now, create the plot with the sorted DataFrame
fig = px.bar(df_by_city_sorted, x='City', y='Sales', title='Sales Sum by City')

# Show the plot
fig.show()

.

.

.

# Business Question n2: Which month has highest number of sold products?

Answer:

In [172]:
# Extract the month from the 'Order Date'
df['Month'] = df['Order Date'].dt.month

# Group by 'Month' and count the number of items
df_by_month = df.groupby('Month').size()

# Find the month with the highest number of items
max_items_month = df_by_month.idxmax()
max_items_count = df_by_month.max()

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Print the result
print(f"The month with the highest number of sold items is: {months[max_items_month - 1]} with {max_items_count} items")

The month with the highest number of sold items is: December with 24984 items


.

Plotting the result:

In [173]:
fig = px.bar(df_by_month, x=df_by_month.index, y=df_by_month.values,
             labels={'x': 'Month', 'y': 'Number of Items'},
             title="Number of Sold Items by Month")

# Updating the x-axis tick labels to display month names
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext= months
    )
)

fig.show()

.

.

# Business Question n3: What time should we display advertisements to increase customer likelihood of buying product?

To reply to this question we are going to agregate data by daily hour.

In [174]:
# Adding an 'Hour' column
df['Hour'] = df['Order Date'].dt.hour

# Count the number of orders for each hour
hourly_order_counts = df.groupby('Hour').size().reset_index(name='Number of Orders')

In [175]:
# Plotting the result
fig = px.line(hourly_order_counts, x='Hour', y='Number of Orders',
              title='Number of Orders by Hour')

# Update x-axis to show every hour
fig.update_xaxes(
    tickmode='array',
    tickvals=list(range(24)),  # Assuming 'Hour' ranges from 0 to 23
    ticktext=[f"{h:02d}:00" for h in range(24)]  # Formatting as '00:00', '01:00', etc.
)

# Show the plot
fig.show()

Answer: We could show the ads before noon and before 7pm.

.

.

# Business Question n4: What items are commonly sold together?

To reply to this question we need to look for duplicates in 'Order ID', to find items that were part of the same order.

In [176]:
# Step 1: Filtering for duplicate 'Order ID's and keeping also the first occurrence
duplicate_order_ids = df[df['Order ID'].duplicated(keep=False)]


# Step 2: Group by 'Order ID' and aggregate products
grouped_orders = duplicate_order_ids.groupby('Order ID')['Product'].apply(list)


# Step 3: Analyze product combinations by counting the frequency of each combination
combination_counts = Counter(grouped_orders.apply(tuple))


# Calculate the maximum length of the item combinations for formatting
max_length = max(len(' + '.join(combination)) for combination, count in combination_counts.most_common(10))

# Display the most common product combinations with right-aligned counts
for combination, count in combination_counts.most_common(10):
    items = ' + '.join(combination)  # Join the items in the combination with ' + '
    print(f"{items.ljust(max_length)} -> {count:>10} times")

iPhone + Lightning Charging Cable                      ->        882 times
Google Phone + USB-C Charging Cable                    ->        856 times
iPhone + Wired Headphones                              ->        361 times
Vareebadd Phone + USB-C Charging Cable                 ->        312 times
Google Phone + Wired Headphones                        ->        303 times
iPhone + Apple Airpods Headphones                      ->        286 times
Google Phone + Bose SoundSport Headphones              ->        161 times
Vareebadd Phone + Wired Headphones                     ->        104 times
Google Phone + USB-C Charging Cable + Wired Headphones ->         77 times
Vareebadd Phone + Bose SoundSport Headphones           ->         60 times
